# Using Numba for GPUs

You may use Numba to generate GPU code from high-level Python functions. Both CUDA and ROC GPUs (NVIDIA and AMD, respectively) are supported, but we are going to focus only on the CUDA capabilities of Numba.

There is an almost one-to-one mapping between the Numba high-level Python abstractions and the different CUDA constructs. This practically means that, as a programmer, you need to take care explicitly of the host/device memory management and you need to be aware of the CUDA programming model.

This demo will teach you to write your first CUDA kernels using Numba. It will cover the basic CUDA programming principles, but it should be enough to kick start you in GPU programming. More specifically, we will cover the following topics:

- Writing a GPU kernel
- Moving data to/from the GPU.
- Spawning a GPU kernel
- Profiling a GPU kernel
- Optimizing memory accesses
- Making use of the shared memory

We will not cover CUDA streams.

## Verify that Numba sees the GPU and understands CUDA

First thing is to check if Numba can detect the GPU. You can achieve this by running the `numba` executable that comes with Numba's installation:

In [ ]:
!numba -s

If this command does not work out of the box for your Numba installation, you may have to set the `CUDA_HOME` environment variable to point to your CUDA installation.

## Writing the first kernel

Here is a vector addition kernel that takes two vectors as input and writes the sum in a third vector:

In [ ]:
import numba.cuda as cuda


@cuda.jit
def _vecadd_cuda(z, x, y):
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


Pretty simple, right? Let's explain each line of this code.

The `@cuda.jit` decorator will compile the following function into a CUDA kernel at runtime. We will see the cost of it later on.

A CUDA kernel in Numba is a Python function that does *not* return a value. This is in accordance with CUDA, where kernel functions are declared `void`. The arguments of the function can be either Numpy arrays or scalars of a Numba recognized type.

CUDA kernels specify the work to be done by a single GPU thread. Due to the massive hardware parallelism available on the GPU, a single GPU thread gets only a tiny portion of work, in this case, the sumation of a single element in the vectors. Since each element is independent from each other, we can safely create as many threads as the elements of the target vectors and spawn them (we will see how later).

Threads on the GPU as organized in groups, called CUDA blocks. There are constraints on the maximum number of threads a block can contain. For the P100 GPUs on Daint, this is 1024. This means that you will need multiple blocks to calculate the sum of large vectors. The blocks that comprise a kernel form the *grid*. Threads inside a block are numbered sequentially and the blocks inside the grid are numbered, too.

In order to obtain the i-th element of the vector given a block size $B$, you would have to calculate the following:

\begin{equation}
i = i_{b}B + i_{t}
\end{equation}

where $i_{b}$ is the block index and $i_{t}$ is the thread index. CUDA provides this information and Numba makes it available, so that the above statement would be written as follows:

```python
i = cuda.blockIdx.x*cuda.blockDim.x + threadIdx.x
```

Blocks and grids can be three dimensional, thus the `.x` attribute in all of these variables (the other dimensions are accessed through the `.y` and `.z` attributes).

Since obtaining the absolute position of a thread in a CUDA is quite common operation, Numba offers the convenience function `grid()`, which essentially does automatically the above calculation:

```python
i = cuda.grid(1)
```

The argument passed to the `grid()` function is the number of dimensions of the grid.

The next three statements in the code simply check that we don't overrun the arrays in case that their dimension is not a multiple of the block size. In this case, some threads of the last block will remain idle:

```python
    N = x.shape[0]
    if i >= N:
        return
```

> The threads inside a block are run in batches of 32 at once, called *warps*. All the threads of the warp execute the same instruction. If the program control flow diverges for some of the threads of the warp, e.g., due to an `if` condition, the branches will be executed sequentially by disabling the non participating threads. This condition is called *warp divergence* and may incur a performance penalty.

Finally, the `z[i] = x[i] + y[i]` computes the actual sum.


## Preparing the data for the GPU

All Numba CUDA kernels operate on data residing on the GPU. That means that the `x`, `y` and `z` arrays must be  transferred from the host (CPU) to the device (accelerator) before calling the CUDA kernel.

Let's create first two vectors on the host:

In [ ]:
import numpy as np


N = 1024
x = np.random.rand(N)
y = np.random.rand(N)

Let's also compute our reference result:

In [ ]:
z_ref = x + y

Now it's time to transfer the vectors to the GPU:

In [ ]:
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)

The `d_x` and `d_y` are Numpy *array-like* objects that have their data mapped in the GPU memory:

In [ ]:
d_x

For the `z` array, we don't need to create it on the host and copy it, since it is essentially an output only array. We can simply allocate it directly on the GPU and copy it out when the kernel finishes.

In [ ]:
d_z = cuda.device_array_like(x)

This will simply allocate an array like `x`, i.e., with the same element type, shape and order, on the GPU.

Data is ready, now it's time to call the kernel!

## Invoking a CUDA kernel

When invoking a CUDA kernel you have to specify the block size to use and the corresponding grid size. Picking the right size of block is not always straightforward, but usually values between 64 and 256 are good enough. 

> The block size has a direct effect on the *occupancy* of each GPU SM, i.e., how much the actual hardware threads of the SM are utilized, and as a result to the occupancy of the whole GPU. Performance-wise, though, it does not have such a big impact. Nvidia provides a nice tool for calculating the occupancy of the GPU, that you can find [here](https://docs.nvidia.com/cuda/cuda-occupancy-calculator/CUDA_Occupancy_Calculator.xls).

For our kernel, we will select a block size of 128 threads.

In [ ]:
block_size = 128

Having decided the block size we need to set up the grid:

In [ ]:
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

Now we are ready to call the kernel!

In [ ]:
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

Both the block and the grid could have been two-dimensional (not in this example), in which case you would just define them as tuples.

## Copying back the results

As mentioned before, kernels operate on GPU data only. We need a way to transfer back to the host the result, which is the `d_z` array. Here's how:

In [ ]:
res = d_z.copy_to_host()

Let's validate the result though to make sure that everything has worked fine:

In [ ]:
assert np.allclose(z_ref, res)

## Putting it all together

For completeness and easy reference, here is the whole vector addition example:

In [ ]:
import numba.cuda as cuda
import numpy as np


@cuda.jit
def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


# Set up the host vectors
N = 1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

### Exercise

> 1. Make the array sufficiently large and time the Numpy version of the sum `x + y`.
> 2. Now time the call to the CUDA kernel with `%timeit -n1 -r1`. What do you see?
> 3. Try timing the CUDA kernels with `%timeit -n1 -r2`. What is happening?

# Understanding the performance of the vecadd kernel

In this section we will see how we can avoid the JIT cost of Numba, how we can measure the performance of the kernel without the `%timeit` magic, how we can use `nvprof`, the CUDA profiler to analyze the performance of the kernel, and finally, we will evaluate the performance of the kernel.

## Avoiding the JIT cost

The previous exercise has shown that Numba will compile the CUDA kernel every time we call our program and, in order to amortize the compilation cost, we need several invocations. We would like to avoid this cost.

Unlike the `@numba.jit` decorator, `@cuda.jit` does not accept a `cache` parameter, that would cache the generated code on the disk and use it on subsequent invocations of the program. Nonetheless, we can force the code generation at import time by supplying a function signature to the `@cuda.jit` decorator that describes the CUDA kernel. This will generate the CUDA code at the time when the decorator processes the function declaration and, therefore, we will avoid the runtime cost of JIT. Let's see how this is done:

In [ ]:
import numba.cuda as cuda
import numpy as np


@cuda.jit('void(Array(float64, 1, "C"), Array(float64, 1, "C"), Array(float64, 1, "C"))')
def _vecadd_cuda(z, x, y):
    '''The CUDA kernel'''
    i = cuda.grid(1)
    N = x.shape[0]
    if i >= N:
        return

    z[i] = x[i] + y[i]


This instructs the Numba runtime to compile the following function into a CUDA kernel (return type `void`) accepting three one-dimensional arrays of `float64` (or `double`) stored in row-major order (C convention). This way, Numba does not have to wait until the `_vecadd_cuda` function is called to figure out the argument types and compile the kernel. It can do this at import time, when it first encounters the function. The downside to that is that you can't call the function with a different type of arguments later. For more details on how you can specify function signatures in Numba, see [here](http://numba.pydata.org/numba-doc/latest/reference/types.html#numba-types).

Let's retry our example now with this version of the kernel.

In [ ]:
# Set up the host vectors
N = 200*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
_vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

### Exercise

> Time the kernel with `%timeit -n1 -r1`. Try to increase the repetitions and experiment with different array sizes. What do you see?

## Measuring the execution time of the kernel

All you see from the previous exercise is the same execution time! What is happening? Actually, you are not measuring the kernel execution time, but rather the kernel launch time. CUDA kernels are launched asynchronously. This means that as soon as you launch the kernel on the GPU, the CPU will continue execution. In this case, it will continue executing and it will block at the statement that copies back the result to the host. 

How do we measure the kernel execution time then? For this, we are going to write a Python [context manager](https://docs.python.org/3.8/reference/datamodel.html?highlight=__getitem__#with-statement-context-managers) so as to measure the execution time of a region in a nice, Pythonic way:

In [ ]:
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)

For more details about context managers, please refer elsewhere. Let's use our timer to time the kernel:

In [ ]:
# Set up the host vectors
N = 200*1000*100
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')

# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

Our timer seems to work fine; we still measure the kernel launch time as with `%timeit`. In order to measure the actual kernel execution time, we need to block the CPU calling thread until the kernel finishes, immediately after we launch the kernel. We can achieve that with `cuda.synchronize()`:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)
    cuda.synchronize()

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

Not bad, the CUDA kernel is 10x faster than the native Numpy kernel.

Before analysing how good or bad this is, let's see an alternative way for measuring the kernel time that actually avoids the use of `cuda.synchronize()`.

## Measuring the kernel execution time with CUDA events

Inserting `cuda.synchronize()` without a reason could slow down your application, since it not only blocks the current CPU thread, but also imposes a synchronization point for all the CUDA streams on the GPU that are currently running in parallel.

> A CUDA stream is essentially a series of sequential operations (data transfers, kernel launches, etc.) that execute on the GPU. Multiple CUDA streams may run independently on the GPU, thus allowing overlapping of operations, such as data transfers and execution of kernels.

To avoid this, but also to obtain a more precise measurement, you can use [CUDA events](http://docs.nvidia.com/cuda/cuda-c-programming-guide/#events). You can imagine CUDA events as milestones associated with timestamps that you can insert between operations in a CUDA stream. Let's how we can adapt our `time_region` context manager to use CUDA events:

In [ ]:
class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start, self._t_end)

To measure a data region with CUDA events you first need to create two events: one for the start and one for the end. You can achieve that with the `cuda.event(timing=True)`. To start counting, you need to call `record()` on the starting event marking the "arrival" to that milestone. Similarly, you call `record()` on the ending event to mark the end of the region. Then you can obtain the elapsed time using the corresponding function as shown in the example above.

Let's rewrite our vector addition example using the CUDA event timers:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
d_x = cuda.to_device(x)
d_y = cuda.to_device(y)
d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
res = d_z.copy_to_host()

# Validate the result
assert np.allclose(x + y, res)

As we can see the execution time obtained is the correct one without having to use `cuda.synchronize()`.

## Assessing the performance of the kernel

The question that arises is how good is the performance that we achieve. Let's inspect further the kernel. Each thread does two `float64` reads from the memory and one write and performs an addition. That means for one floating operation, the kernel must transfer to/from memory 24 bytes from the main memory. This gives us an  *arithmetic intensity* or *flop:byte ratio* of 0.0417. The lower this ratio is for a computational kernel, the more likely will be that the kernel is memory bandwidth bound. As the ratio increases, the kernel tends to be more compute bound. The theory behind the arithmetic intensity is covered by the *Roofline* performance model, which is outside the scope of this tutorial. For the moment, let's compute two performance metrics, the `Gflop/s` achieved by the kernel and the data transfer rate to/from memory:

In [ ]:
print(f'Performance: {1.e-9*N/t_kernel.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_kernel.elapsed_time()} GB/s')

Let's compute the same for the NumPy kernel:

In [ ]:
print(f'Performance: {1.e-9*N/t_ref.elapsed_time()} Gflop/s')
print(f'Transfer rate: {1.e-9*3*N*8/t_ref.elapsed_time()} GB/s')

As you can, the GPU can deliver more than 10x bandwidth compared to the CPU. Checking the [datasheet](https://www.nvidia.com/content/dam/en-zz/Solutions/Data-Center/tesla-p100/pdf/nvidia-tesla-p100-datasheet.pdf) of the NVIDIA P100 GPU, we can see that the peak nominal memory bandwidth is 732 GB/s, meaning that our kernel utilizes 70% of the peak bandwidth.

> Achieving the nominal peak memory bandwidth is usually not possible with real-life computational kernels, even with very low arithmetic intensity. For this reason, we tend to cite the *effective memory bandwidth*, which is obtained by benchmarks like the one presented in this tutorial. In fact, the effective memory bandwidth of the P100 GPUs is at ~550 GB/s, which essentially shows that the vector addition kernel's performance is optimal. For the Haswell CPUs on the host, the effective memory bandwidth is ~50 GB/s.

## Understanding the data transfer overhead

So far we have only focused on the performance of the kernel. There is still a quite important topic we have not yet addressed. CUDA kernels require that the data they operate on is located on the device and we need to move that data there from the host. What is the cost of this data movement? Let's time our benchmark code:

In [ ]:
# Set up the host vectors
N = 200*1000*1000
x = np.random.rand(N)
y = np.random.rand(N)

# Copy and allocate data on the device
with time_region_cuda() as t_copyin:
    d_x = cuda.to_device(x)
    d_y = cuda.to_device(y)

with time_region_cuda() as t_create:
    d_z = cuda.device_array_like(x)

# Set up the kernel invocation
block_size = 128
num_blocks = N // block_size
if N % block_size:
    num_blocks += 1

# Call the kernel
with time_region_cuda() as t_kernel:
    _vecadd_cuda[num_blocks, block_size](d_z, d_x, d_y)

with time_region() as t_ref:
    z = x + y

print(f'CUDA kernel time: {t_kernel.elapsed_time()} s')
print(f'Numpy time:       {t_ref.elapsed_time()} s')


# Copy back the result to the host
with time_region_cuda() as t_copyout:
    res = d_z.copy_to_host()

print(f'Copyin time:  {t_copyin.elapsed_time()}')  
print(f'Create time:  {t_create.elapsed_time()}')    
print(f'Copyout time: {t_copyout.elapsed_time()}')    


# Validate the result
assert np.allclose(x + y, res)

The data copy times are quite important! In fact, if we include these in the total execution time of the GPU version, the CPU version becomes more than 8x faster! Data transfers is the No. 1 optimization that you should do when programming for the GPUs. You must minimize the data transfers to/from GPU by keeping the necessary data on the GPU for as long as it is needed.

Before closing this discussion, let's see how fast is the data moved over to the GPU:

In [ ]:
print(f'Copyin rate: {1e-9*2*N*8/t_copyin.elapsed_time()} GB/s')

This is bound by the data rate of the PCI 16x bus where the GPU is attached to and, it is indeed way too slower than the main memory bandwidth of modern processors.

Interestingly, the copyout data rate seems to be much slower:

In [ ]:
print(f'Copyout rate: {1e-9*N*8/t_copyout.elapsed_time()} GB/s')

By default, memory allocated on the host is pageable. This means that it can be moved by the OS kernel to a secondary storage device if there is not enough memory available on the system. This can incur a significant performance penalty, especially if you write on freshly allocated memory (as it happens in our example). You can avoid this overhead by using *page-locked* or *pinned* memory. This memory cannot be paged out and it is physically resident on the memory device. CUDA gives you the opportunity to use pinned memory and Numba allows you to create pinned ndarrays using the [cuda.pinned_array()](http://numba.pydata.org/numba-doc/latest/cuda-reference/memory.html#numba.cuda.pinned_array) function.

> In order to keep track of which memory pages are resident on the physical memory and which are not, the OS kernel maintains a special data structure called *page table*. When you allocate memory on the host, the OS kernel simply creates a virtual memory mapping and it does not allocate any physical page. As soon as you start writing to the memory area you have been allocated, it will look for the page in its page tables and if not found, a *page fault* will be raised and then the kernel will have to physically allocate the missing memory page and update its page tables.

Let's rewrite the copyout part using pinned memory:

In [ ]:
res = cuda.pinned_array(N)
with time_region_cuda() as t_pinned:
    d_z.copy_to_host(res)
    
assert np.allclose(x + y, res)
print(f'Copyout data rate (pinned): {1e-9*N*8/t_pinned.elapsed_time()} GB/s')

Notice how much the performance has improved. It is now even better than the copyin operation.

However, pinned memory does not come without a cost. Since pinned pages cannot be paged out, they will stay on the physical memory, increasing the memory pressure and, finally, the effective memory consumption of the code. For memory hungry applications, this can be a problem.

### Exercise

> Apply the pinned memory technique also to the input arrays `x` and `y`.

## Profiling the CUDA code

In this simple example of vector addition we assessed the performance and identified the bottlenecks ourselves, by analyzing the code structure and reasoning about it. In more complex codes or codes that you are not very familiar with, it would be good if this analysis could be done by a dedicated tool. Not to be misunderstood, understanding the code structure and its memory and compute requirements is essential for optimizing it in any case, but using a *performance profiler* is very handy for analyzing the performance bottlenecks, for helping you prioritizing your optimization targets and for understanding how much room for improvement exists.

NVIDIA provides [Nsight](https://developer.nvidia.com/nsight-visual-studio-edition) and `nvprof` for profiling CUDA code. In this tutorial, we are going to use `nvprof` and the [Nvidia Visual Profiler](https://developer.nvidia.com/nvidia-visual-profiler) to inspect the results.

> You may install the Nvidia Visual Profiler on your personal computer and visualize the performance results, even if you don't have a GPU.

The `src/vecadd.py` file contains the vector addition example as we have finally presented it here. Let's do a basic profing:

```bash
srun -C gpu nvprof -o vecadd.nvprof python src/vecadd.py $((200*1000*1000))
```

> If you want to profile your code with `nvprof` you should call `cuda.profile_stop()` at the end of your program.

This profiling provides basic information about the data transfers and the execution time of the different kernels. It is the first step you need to take, because this will show you how much time you spend on transferring data to/from the device and which kernels are the most time consuming.

Notice how much is the overhead of the data transfers as well as that of the pinned memory allocation of the `res` array. Placing your cursor on top of any of the regions in the timeline you can see more information. In this case, I have highlighted the copy-to-host operation, where you can see that the target memory on the host is pinned and the data rate is exactly as the one we have calculated above.

As soon as you have addressed the data transfer issues, the next step is to identify the performance bottlenecks in the most time consuming kernels. To do that, you need more detailed information that reflects hardware events happening on the GPU (e.g., instructions executed, data transferred from the main memory, use of caches etc.). To achieve this you have to pass the `--analysis-metrics` option to `nvprof` as follows:

```bash
srun -C gpu nvprof -o vecadd.detailed.nvprof --analysis-metrics python src/vecadd.py $((200*1000*1000))
```

As you will notice this command incurs quite of an overhead. The reason behind that is that there are only a few hardware performance monitoring registers on the GPU, so in order for `nvprof` to collect all the necessary performance metrics, it will have to rerun the kernel several times. From this type of analysis you can obtain the actual memory bandwidth consumed by your kernel.

As expected the memory bandwidth consumption is red highlighted since it is the performance limiting factor, as we have also calculated manually. The bandwidth consumption reported here is 557 GB/s, which is quite close with our measurements based on the algorithm details.

This concludes our discussion on measuring and analyzing the performance of a CUDA program written with Numba.

# Understanding the memory performance of the GPU

In this section we are going to investigate a crucial aspect of the memory locality on the GPUs. It should be perceived in a slightly different way than on the CPUs. To demonstrate this, we will use BLAS matrix-vector kernel using all the tricks we have learned so far. The threads of the GPU operate row-wise in the input matrix, each one taking care of a single row to compute:

In [ ]:
import numba
import numba.cuda as cuda
import numpy as np
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)


class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start,
                                                              self._t_end)


@cuda.jit('void(float64, Array(float64, 2, "C"), Array(float64, 1, "C"), '
          'float64, Array(float64, 1, "C"))')
def _gemv_cuda(alpha, A, x, beta, y):
    i = cuda.grid(1)
    N, M = A.shape
    if i >= N:
        return

    prod = 0.0
    for j in range(M):
        prod += A[i, j]*x[j]

    y[i] = alpha*prod + beta*y[i]


def gemv_gpu(alpha, A, x, beta, y):
    # Works only for square matrices
    N = A.shape[0]
    with time_region_cuda() as t_xfer:
        d_A = cuda.to_device(A)
        d_x = cuda.to_device(x)
        d_y = cuda.to_device(y)
        y_ret = cuda.pinned_array(N)
        
    block_size = 128
    num_blocks = N // block_size
    if N % block_size:
        num_blocks += 1

    with time_region_cuda() as t_kernel:
        _gemv_cuda[num_blocks, block_size](alpha, d_A, d_x, beta, d_y)

    with time_region_cuda(t_xfer.elapsed_time()) as t_xfer:
        d_y.copy_to_host(y_ret)

    print(f'  CUDA transfer overheads: {t_xfer.elapsed_time()}')
    print(f'  CUDA kernel time: {t_kernel.elapsed_time()}')
    print(f'  Consumed memory bandwidth: {1e-9*8*N*(N+2)/t_kernel.elapsed_time()} GB/s')
    return y_ret

N = 1024*16
A = np.random.rand(N, N)
x = np.random.rand(N)
y_orig = np.ones(N)
alpha = 0.2
beta = 1

with time_region() as t_gpu:
    y = gemv_gpu(alpha, A, x, beta, y_orig)

with time_region() as t_ref:
    y_ref = alpha*(A @ x) + beta*y_orig
    
    
cuda.profile_stop()

print(f'Total time (GPU): {t_gpu.elapsed_time()} s')
print(f'Total time (CPU): {t_ref.elapsed_time()} s')

assert np.allclose(y, y_ref)

### Exercise

> Increase the array size and record the CUDA kernel performance time. How much faster is it compared to the CPU version?

As you might have noticed already, the GPU kernels is only about 2-3x faster than the CPU version. Not as good it has been with the vector addition kernel. Is this expected, is this how it should be? Let's look into the kernel in more detail.

The kernel needs to read the whole matrix $A$ and the vectors $x$ and $y$, i.e., $8(N^2 + 2N)$ bytes need to be transferred to/from main memory in total. At the same time, the kernel performs $2N^2 + 3N$ floating point operations in total. This leads to an arithmetic intensity or flop:byte ratio equals to $\frac{2N(N+
\frac{3}{2})}{8N(N+2)} \approx 0.25$. This ratio is much higher than that for the vector addition kernel, but it is very low to make the kernel compute bound.

> Given the nominal peak double precision performance (5.3 Tflop/s) and the nominal peak memory bandwidth of the P100 GPUs (732 GB/s), a kernel would need a flop:byte ratio of at least 7.24, so as to be compute bound.

So, theoretically, we should be approaching the effective memory bandwidth limit of the device, but we only achieve 1/4 of it. The CPU kernel on the other hand seems to hit its memory bandwidth limit:

In [ ]:
print(f'CPU memory bandwidth consumed: {1e-9*8*N*(N+2)/t_ref.elapsed_time()}')

What is going on? The following figure shows the memory layout of the $A$ matrix, the $x$ and $y$ vectors and how the CUDA threads are arranged for the computation. The matrix is stored in the default, row-major, order and each thread is assigned a row of the matrix.

> Assigning a column to each thread would require a reduction operations across all the threads on the device. Global reduction on GPUs is not straightforward to achieve, because there can't be any sort of global synchronization. You would have to write multiple kernels to achieve a global reduction. For reduction operations, you should use the `@cuda.reduce` decorator (see [here](http://numba.pydata.org/numba-doc/latest/cuda/reduction.html) for more information).

In this arrangement subsequent threads access non-contiguous memory, but each thread accesses memory sequentially. This would be almost ideal on the CPU: perfect read access on both the $A$ matrix and the $x$ vector. What is "wrong" with the GPUs?

> For the CPUs the ideal is to assign a chunk of rows to each thread, because the above arrangement would lead to false sharing.

Recall from our discussion on CUDA blocks in the beginning how threads are executed on the GPU. Threads are executed in batches of 32, called warps, all of them executing the same instruction. In our example, that means that each thread in the warp will access memory allocated for the $A$ matrix in $8N$ bytes strides.

The global memory on the GPU is organized in 256-byte memory segments and can be accessed in transactions of 32, 64 or 128 bytes. If all the threads of a warp access the same memory segments, a maximum of two memory transactions will be performed in order to fetch the values required by all the threads of the warp. This is called *memory coalescing* in CUDA's terminology. In our example, a warp needs to fetch $32\times 8 = 256$ bytes, i.e., 2 memory transactions of 128 bytes each ideally. However, due to the row-major layout of the matrix $A$, each thread does a separate 32 byte transaction, generating $32\times 32 = 1024$ bytes memory traffic per warp, which is four times more than the ideal! How much of the memory bandwidth did our code utilize?

### Exercise

> Change the memory layout of matrix $A$ to column major (or Fortran in NumPy's nomenclature) and measure again the performance. How close is it to the effective memory bandwidth limit? Hints: (a) `numpy.random.rand()` does not accept an `order` argument. Use the `asarray()` function to create matrix $A$ from the result of `numpy.random.rand()` and with the desired order. (b) Make sure to adjust order in the JIT function signature.


## Taking advantage of the shared memory

On every GPU multiprocessor chip (SM), there is an on-chip memory called *shared memory*. This can either act as L1 data cache memory (default mode) or it can serve as programmable scratchpad at the disposal of the programmer. This memory is shared among all the threads of a CUDA block. We are going to modify our matrix-vector kernel to make use of it. If you inspect closer the algorithm, you will see that although there is no temporal locality in accesses to the matrix $A$, the the vector $x$ is reused $N$ times. For this reason, we are going to cache the vector $x$ manually into the shared memory.

We essentially process the matrix with a sliding window. The threads of a block (still 1D) undertake a double role: first, they fetch the $x$ vector elements into the shared memory of the block and, second, they perform the multiplication and the reduction to a local register variable. As soon as they process the full row, the store this value back to vector $y$. Here is the implementation in Numba:

In [ ]:
import numba
import numba.cuda as cuda
import numpy as np
import time


class time_region:
    def __init__(self, time_offset=0):
        self._time_off = time_offset

    def __enter__(self):
        self._t_start = time.time()
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end = time.time()

    def elapsed_time(self):
        return self._time_off + (self._t_end - self._t_start)


class time_region_cuda:
    def __init__(self, time_offset=0, cuda_stream=0):
        self._t_start = cuda.event(timing=True)
        self._t_end = cuda.event(timing=True)
        self._time_off = time_offset
        self._cuda_stream = cuda_stream

    def __enter__(self):
        self._t_start.record(self._cuda_stream)
        return self

    def __exit__(self, exc_type, exc_value, traceback):
        self._t_end.record(self._cuda_stream)
        self._t_end.synchronize()

    def elapsed_time(self):
        return self._time_off + 1.e-3*cuda.event_elapsed_time(self._t_start,
                                                              self._t_end)
BLOCK_SIZE = 128
    
@cuda.jit('void(float64, Array(float64, 2, "F"), Array(float64, 1, "F"), '
          'float64, Array(float64, 1, "F"))')
def _gemv_cuda_shared(alpha, A, x, beta, y):
    i = cuda.grid(1)
    N, M = A.shape
    if i >= N:
        return

    lx = cuda.shared.array(shape=BLOCK_SIZE, dtype=numba.float64)
    bsize = cuda.blockDim.x
    tid = cuda.threadIdx.x
    num_blocks = cuda.gridDim.x

    prod = 0.0
    for b in range(num_blocks):
        lx[tid] = x[tid + b*bsize]
        cuda.syncthreads()

        for j in range(BLOCK_SIZE):
            prod += A[i, j + b*bsize]*lx[j]

        cuda.syncthreads()

    y[i] = alpha*prod + beta*y[i]

def gemv_gpu(alpha, A, x, beta, y):
    # Works only for square matrices
    N = A.shape[0]
    with time_region_cuda() as t_xfer:
        d_A = cuda.to_device(A)
        d_x = cuda.to_device(x)
        d_y = cuda.to_device(y)
        y_ret = cuda.pinned_array(N)
        
    num_blocks = N // BLOCK_SIZE
    if N % BLOCK_SIZE:
        num_blocks += 1

    with time_region_cuda() as t_kernel:
        _gemv_cuda_shared[num_blocks, BLOCK_SIZE](alpha, d_A, d_x, beta, d_y)

    with time_region_cuda(t_xfer.elapsed_time()) as t_xfer:
        d_y.copy_to_host(y_ret)

    print(f'  CUDA transfer overheads: {t_xfer.elapsed_time()}')
    print(f'  CUDA kernel time: {t_kernel.elapsed_time()}')
    print(f'  Consumed memory bandwidth: {1e-9*8*N*(N+2)/t_kernel.elapsed_time()} GB/s')
    return y_ret

N = 1024*16
A = np.asarray(np.random.rand(N, N), order='F')
x = np.random.rand(N)
y_orig = np.ones(N)
alpha = 0.2
beta = 1

with time_region() as t_gpu:
    y = gemv_gpu(alpha, A, x, beta, y_orig)

with time_region() as t_ref:
    y_ref = alpha*(A @ x) + beta*y_orig
    
    
cuda.profile_stop()

print(f'Total time (GPU): {t_gpu.elapsed_time()} s')
print(f'Total time (CPU): {t_ref.elapsed_time()} s')

assert np.allclose(y, y_ref)

The kernel is more complex now, but let's take it step by step. The `cuda.shared.array(shape=BLOCK_SIZE, dtype=numba.float64)` statement allocates the part of the $x$ vector that is stored in shared memory. When allocating an array in shared memory, the shape must be "constant." Although constants don't exist in Python, in the context of Numba, this means that at the time the CUDA kernel is compiled, its value must be known. You may not use for example `cuda.blockDim.x`, since this is not known at the time the kernel is compiled. Strangely, though, you are not allowed to do `shape=2*CONST`, where `CONST` is a "constant" as defined above.

> In pure CUDA, you may write a kernel with an unknown shared memory array by declaring it `extern` and passing its size at the invocation of the kernel. In Numba, you can't do that.

The main work of the algorithm is done in the following loop:

```python
    prod = 0.0
    for b in range(num_blocks):
        lx[tid] = x[tid + b*bsize]
        cuda.syncthreads()

        for j in range(BLOCK_SIZE):
            prod += A[i, j + b*bsize]*lx[j]

        cuda.syncthreads()
        
    y[i] = alpha*prod + beta*y[i]
```

The outer loop iterates over the blocks in the $j$ direction and each thread fetches an element from $x$ and places it in the shared memory buffer. Before proceeding to the actual computation, we need to make sure that the shared buffer has been populated fully, thus we insert a thread barrier with `cuda.syncthreads()`. This barrier affects *only* the threads of a single block. We then move to the actual computation where each thread computes a row inside the sliding widing. At the end we have to synchronize again, since we need to make sure that the `lx` buffer is consumed fully, before we start refilling it in the next iteration. Finally, after the threads have gone through all the blocks in the $j$ direction, they compute the final result in $y$.

### Quiz

> If the block size was 32, could we omit `cuda.syncthreads()` and why?

### Exercise

> 1. Compare the performance of this kernel with the standard kernel with column-major layout that we presented before. Which one is faster?
> 2. Run `nvprof` on the `src/matvec.py` and try to understand what could be the overhead.

As you might have noticed, this kernel is 20% slower than the seemingly naive one. In fact, we shouldn't have expected any improvement whatsoever, since the "naive" kernel already hits the memory bandwidth limit. But why manually caching didn't work? When the shared memory is not used as a scratchpad memory, it functions as a standard L1 cache. So, essentially, the caching of $x$ did happen implicitly in the original version. The only thing we have achieved with the shared memory version is to introduce an additional computational overhead due to the sliding window; `nvprof` shows that. Truth must be said though, that in the early generations of NVIDIA GPUs, where shared memory did not function as a standard cache, this optimization did give you additional performance.

So when is shared memory beneficial? It is only when you have a kernel with a high arithmetic intensity, which implies some significant temporal locality in the memory accesses. The most prominent example of such a kernel is the matrix-matrix multiplication, which has an arithmetic intensity at the order of $N$. In such a case, you could use the shared memory to implement the tiling algorithm. An example implementation with Numba an be found [here](http://numba.pydata.org/numba-doc/latest/cuda/examples.html?highlight=matrix%20matrix#matrix-multiplication). Another candidate user of shared memory could be layered stencil computations, where you could cache the intermediate layers.